In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'
# %pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)
from syft import autocache

In [ ]:
node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True)

In [ ]:
domain_client = node.login(email="info@openmined.org", password="changethis")

In [ ]:
from syft.client.client import SyftClient
from syft.store.blob_storage import BlobDeposit
from syft.types.blob_storage import CreateBlobStorageEntry
from syft.types.syft_object import SyftObject

In [ ]:
def allocate_object(client: SyftClient, obj: SyftObject) -> BlobDeposit:
    create_blob_storage_entry = CreateBlobStorageEntry.from_obj(obj)
    return client.api.services.blob_storage.allocate(create_blob_storage_entry)

In [ ]:
user_object = domain_client.users[0]
user_object

In [ ]:
# write/deposit object
blob_deposit = allocate_object(domain_client, user_object)
write_result = blob_deposit.write(sy.serialize(user_object, to_bytes=True))
write_result

In [ ]:
# read/retrieve object
blob_retrieval = domain_client.api.services.blob_storage.read(blob_deposit.blob_storage_entry.id)
user_object_read = blob_retrieval.read()
user_object_read